In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, confusion_matrix, classification_report

In [3]:
df = pd.read_csv('/home/jensk/code/JensKlug/zillows_real_estate/raw_data/HouseTS.csv')

In [4]:
df[['zipcode', 'date']]

,zipcode,date
0,30002,2012-03-31
1,30002,2012-04-30
2,30002,2012-05-31
3,30002,2012-06-30
4,30002,2012-07-31
...,...,...
884087,34698,2023-08-31
884088,34698,2023-09-30
884089,34698,2023-10-31
884090,34698,2023-11-30


In [5]:
df_sorted = df.sort_values(['zipcode', 'date']).reset_index(drop=True)
df_sorted[['zipcode', 'date']]

,zipcode,date
0,1431,2012-03-31
1,1431,2012-04-30
2,1431,2012-05-31
3,1431,2012-06-30
4,1431,2012-07-31
...,...,...
884087,98686,2023-08-31
884088,98686,2023-09-30
884089,98686,2023-10-31
884090,98686,2023-11-30


In [6]:
type(df_sorted['date'][0]), df_sorted['date'][0]

(str, '2012-03-31')

In [7]:
df_sorted['date'] = pd.to_datetime(df_sorted['date'])
df_sorted['date']

0        2012-03-31
1        2012-04-30
2        2012-05-31
3        2012-06-30
4        2012-07-31
            ...    
884087   2023-08-31
884088   2023-09-30
884089   2023-10-31
884090   2023-11-30
884091   2023-12-31
Name: date, Length: 884092, dtype: datetime64[ns]

In [8]:
type(df_sorted['date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [9]:
df_sorted[['zipcode','date']]

,zipcode,date
0,1431,2012-03-31
1,1431,2012-04-30
2,1431,2012-05-31
3,1431,2012-06-30
4,1431,2012-07-31
...,...,...
884087,98686,2023-08-31
884088,98686,2023-09-30
884089,98686,2023-10-31
884090,98686,2023-11-30


In [10]:
def split_train_predict(df):
    # Ensure it's sorted by zipcode and date
    df = df.sort_values(['zipcode', 'date'])

    # Group by zipcode
    train_list = []
    predict_list = []

    for zipcode, group in df.groupby('zipcode'):
        train_part = group.iloc[:-3]
        predict_part = group.iloc[-3:]

        train_list.append(train_part)
        predict_list.append(predict_part)

    # Concatenate the results
    df_train = pd.concat(train_list).reset_index(drop=True)
    df_predict = pd.concat(predict_list).reset_index(drop=True)

    return df_train, df_predict

# Apply the split
df_train, df_predict = split_train_predict(df_sorted)

# Optional: verify
print("Train set:", df_train.shape, 'Type', type(df_train))
print("Predict set:", df_predict.shape, 'Type', type(df_predict))

Train set: (865414, 39) Type <class 'pandas.core.frame.DataFrame'>
Predict set: (18678, 39) Type <class 'pandas.core.frame.DataFrame'>


In [12]:
grouped_train = df_train.groupby(['zipcode'])
grouped_predict = df_predict.groupby(['zipcode'])

In [13]:
ts_train = pd.DataFrame()

ls = [1,3,6,12]

i = 1

#for i in ls:
ts_train[f'target_profitable_{i}m'] = (grouped_train['price'].shift(-i) > 1.02 * df_train['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_train['zipcode_obj'] = grouped_train['zipcode'].shift(0).astype('object') # but remains still an integer...
numeric_column_names = df_train.select_dtypes(include='number').columns # drop the lags of zipcode-> this in Jessica's code.


for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)

ts_train = ts_train.drop(columns=['zipcode_1m', 'zipcode_2m', 'zipcode_3m'])

/tmp/ipykernel_24840/1700554156.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
/tmp/ipykernel_24840/1700554156.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
/tmp/ipykernel_24840/1700554156.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

In [18]:
ts_train

,target_profitable_1m,zipcode_obj,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,median_ppsf_2m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
0,0,1431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1431,89000.0,NaN,NaN,252250.0,NaN,NaN,65.441176,NaN,...,NaN,2933.0,NaN,NaN,1493.0,NaN,NaN,195148.086474,NaN,NaN
2,0,1431,124000.0,89000.0,NaN,229500.0,252250.0,NaN,120.984975,65.441176,...,NaN,2933.0,2933.0,NaN,1493.0,1493.0,NaN,194446.137500,195148.086474,NaN
3,0,1431,177500.0,124000.0,89000.0,207450.0,229500.0,252250.0,120.984975,120.984975,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,194580.490265,194446.137500,195148.086474
4,0,1431,217000.0,177500.0,124000.0,203750.0,207450.0,229500.0,121.340580,120.984975,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,195081.437213,194580.490265,194446.137500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865409,0,98686,506000.0,499900.0,499900.0,502560.0,520220.0,570445.0,285.245902,268.612521,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,553637.387867,552276.246723,552746.639171
865410,0,98686,523227.5,506000.0,499900.0,474900.0,502560.0,520220.0,285.711767,285.245902,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,555511.734585,553637.387867,552276.246723
865411,0,98686,530900.0,523227.5,506000.0,520000.0,474900.0,502560.0,287.859825,285.711767,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,558047.224949,555511.734585,553637.387867
865412,0,98686,515000.0,530900.0,523227.5,527450.0,520000.0,474900.0,305.216752,287.859825,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,561259.870706,558047.224949,555511.734585


In [15]:
ts_test = pd.DataFrame()

#for i in ls:
ts_test[f'target_profitable_{i}m'] = (grouped_predict['price'].shift(-i) > 1.02 * df_predict['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_test['zipcode'] = grouped_predict['zipcode'].shift(0)# .astype('object') # but remains still an integer...
numeric_column_names = df_predict.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_test[f'{feature}_{lag}m'] = grouped_predict[f'{feature}'].shift(lag)

/tmp/ipykernel_24840/3971834618.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_test[f'{feature}_{lag}m'] = grouped_predict[f'{feature}'].shift(lag)
/tmp/ipykernel_24840/3971834618.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_test[f'{feature}_{lag}m'] = grouped_predict[f'{feature}'].shift(lag)
/tmp/ipykernel_24840/3971834618.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at on

In [16]:
ts_test

,target_profitable_1m,zipcode,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,median_ppsf_2m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
0,0,1431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1431,462500.0,NaN,NaN,450000.0,NaN,NaN,219.758478,NaN,...,NaN,3170.0,NaN,NaN,1541.0,NaN,NaN,429997.675464,NaN,NaN
2,0,1431,469000.0,462500.0,NaN,398888.0,450000.0,NaN,200.601023,219.758478,...,NaN,3170.0,3170.0,NaN,1541.0,1541.0,NaN,431098.634112,429997.675464,NaN
3,0,1432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1432,485000.0,NaN,NaN,535000.0,NaN,NaN,323.688969,NaN,...,NaN,7887.0,NaN,NaN,3905.0,NaN,NaN,502059.557636,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18673,0,98685,640000.0,NaN,NaN,599475.0,NaN,NaN,297.304440,NaN,...,NaN,29456.0,NaN,NaN,11629.0,NaN,NaN,577247.107610,NaN,NaN
18674,0,98685,640000.0,640000.0,NaN,599000.0,599475.0,NaN,291.176471,297.304440,...,NaN,29456.0,29456.0,NaN,11629.0,11629.0,NaN,577994.879622,577247.107610,NaN
18675,0,98686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18676,0,98686,525000.0,NaN,NaN,504450.0,NaN,NaN,306.527550,NaN,...,NaN,19904.0,NaN,NaN,7787.0,NaN,NaN,568266.780424,NaN,NaN
